In [1]:
import torch 
from transformers import DistilBertTokenizer, DistilBertModel
import nltk
from nltk.corpus import stopwords

from io import StringIO
from html.parser import HTMLParser
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import json
 
nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',truncation_side='left',truncation=True)
tokenizer.add_tokens(list(open('latex-vocabulary/latex_symbols.txt','r')))

C:\Users\Anna\AppData\Local\Temp\ipykernel_18788\3911523092.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


13855

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print(x)
else:
    print("MPS device not found.")
    device = torch.device("cpu")

MPS device not found.


In [3]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def get_latex_from_alt(context):
    strip_deliminators = lambda latex: latex.replace('$','').replace('\\[','').replace('\\]','')

    context_soup = BeautifulSoup(context)
    latex_images = context_soup.find_all('img')
    for image in latex_images:
        image.replace_with(strip_deliminators(image['alt']))
    # return [strip_deliminators(image['alt']) for image in latex_images]
    return str(context_soup)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_stopwords(text):
    filtered_text = [w for w in text.split() if w.lower() not in nltk_stopwords]
    return " ".join(filtered_text)

In [4]:
problems = json.load(open('amc_8_problems_with_sol.json'))

In [16]:
problem_text = strip_tags(get_latex_from_alt(problem))
solutions_text = strip_tags(get_latex_from_alt(" ".join([solution for solution in json.loads(solutions_list) if 'http' not in solution])))
choices_text = " ".join(json.loads(choices))
training_text = " ".join([problem_text, solutions_text, choices_text])

NameError: name 'problem' is not defined

In [5]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self, num_classes):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        return self.classifier(pooler)

In [6]:
model = DistilBERTClass(num_classes=5)
model.to(device)
model = torch.load('top-level-with-solutions-distilbert-amc10-2019-2022.pt',map_location=device)

def outputs(input_string):
    inputs = tokenizer.encode_plus(
            input_string,
            None,
            add_special_tokens=True,
            max_length=256,
            pad_to_max_length=True,
            return_tensors="pt").to(device)
    fin_outputs = []
    with torch.no_grad():
        outputs = model(**inputs)
    fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    outputs = np.array(fin_outputs) >= 0.5
    return [[1 if value else 0 for value in output] for output in outputs][0]

output = "";
for year in range(2015,2025):
    if year==2021:
        continue
    for problem_num in range(1,26):
        problem_ind = str(year)+" AMC 8 #"+str(problem_num)
        if (problem_ind=="2019 AMC 8 #5" or problem_ind=="2019 AMC 8 #12" or problem_ind=="2022 AMC 8 #4" or problem_ind=="2022 AMC 8 #10" or problem_ind=="2022 AMC 8 #14" or problem_ind=="2023 AMC 8 #2"):
            continue
        problem_text = strip_tags(get_latex_from_alt(problems[problem_ind]["problem"]))
        solutions_text = strip_tags(get_latex_from_alt(" ".join([solution for solution in problems[problem_ind]["solutions"] if 'http' not in solution])))
        choices_text = " ".join(problems[problem_ind]["choices"])
        combined_text = remove_stopwords(" ".join([problem_text, solutions_text, choices_text]))
        assignment = outputs(combined_text)
        if assignment==[0,0,0,0,0]:
            assignment[0] = 1;
        output+=problem_ind+": "+str(assignment)+"\n"

        with open('outputs_file.txt', 'w') as f:
            f.write(output)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Anna\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


AttributeError: 'DistilBertModel' object has no attribute '_use_flash_attention_2'